<a href="https://colab.research.google.com/github/njainds/Colab_notebooks/blob/master/CCP_ICD10_Searchengine/DataPrep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Data Preparation
#Cleaning: Spellchecks, lemma
#Create embedding matrix, missing embedding imputation

# Build tokenizer and vocab
# create test and train with sequence

# Builf train and evaluate

# Save model for offline scoring on test

In [0]:
# Data preparation

In [1]:
import torch
print("Name of GPU : {}".format(torch.cuda.get_device_name(0)))
print("# of GPU : {}".format(torch.cuda.device_count()))


Name of GPU : Tesla K80
# of GPU : 1


In [2]:
from fastai.text import *
import html
import sklearn
import torch.tensor as T
from sklearn.model_selection import train_test_split

#from fastai.learner import *

import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

import pandas as pd
import string
import time
import random
import pandas as pd
import numpy as np
import gc
import re
import torch
from torchtext import data
import spacy
from tqdm import tqdm_notebook, tnrange
from tqdm.auto import tqdm
import zipfile
import gzip

tqdm.pandas(desc='Progress')
from collections import Counter
from textblob import TextBlob
from nltk import word_tokenize
import torch
import torch.nn as nn
import random
import keras
import numpy as np
from keras.layers import Conv1D, Conv2D
from keras import optimizers
import os
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import nltk

# cross validation and metrics
from  sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection,linear_model,metrics
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from torch.optim.optimizer import Optimizer
from gensim.models.keyedvectors import KeyedVectors

Using TensorFlow backend.


In [3]:
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/My Drive/kaggle_toxic_comments/ulmfit"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
data	     data_save.pkl  LM_data.csv  test_X.csv
data_cl.pkl  export.pkl     models	 train_X.csv


In [5]:
df = pd.read_csv('/content/drive/My Drive/CCP-ICDsearch/desc-icd9-modeldata/model_data.csv')
#df.head()
# removed duplicated rows
print(df.shape)
df.drop_duplicates(inplace=True)
print(df.shape)

(16981, 2)
(8705, 2)


In [0]:
#misspell correction
def build_vocab(texts):
    sentences = texts.apply(lambda x: re.split(' |;|,|\*|\n|[(]|[)]',x))
    dfvocab = {}
    for sentence in sentences:
        for word in sentence:
            try:
                dfvocab[word] += 1
            except KeyError:
                dfvocab[word] = 1
    return dfvocab
dfvocab = build_vocab(df['desc'])

#len(vocab.keys()) 3630

model = KeyedVectors.load_word2vec_format('/content/drive/My Drive/CCP-ICDsearch/wikipedia-pubmed-and-PMC-w2v.bin', binary=True)
misspells=[]
for k in list(dfvocab.keys()):
    if k in model.vocab:
        continue
    else:
        misspells.append(k)

#misspells
#[i for i in misspells if '/' in i]

In [0]:
#cleaning
#replcae + and +ve by ' positive'
#replcase r. and l. with right and left
#replcae int/int with int#int
#split with /:,|()+\-
#misspells dict


In [0]:
#np.save('/content/drive/My Drive/CCP-ICDsearch/desc-icd9-modeldata/mispell_dict.npy',mispell_dict)
#del mispell_dict
#mispell_dict = np.load('/content/drive/My Drive/CCP-ICDsearch/desc-icd9-modeldata/mispell_dict.npy').item()

In [0]:
mispell_dict = np.load('/content/drive/My Drive/CCP-ICDsearch/desc-icd9-modeldata/mispell_dict.npy').item()
puncts = ['/','\',':','|','(',')','+','-','*','!']

def clean_text(x):
    x = x.replace(r'+ve',' positive ')
    x = x.replace(r'+',' positive ')
    x = re.sub(r"\b([l][.])", "left ", x)
    x = re.sub(r"\b([r][.])", "right ", x)
    x = ((''.join('#'+i+'#'  if i.isdigit() else i for i in x)).replace('#/#','<>')).replace('#','')
    x = x.replace(r"\n", r" ")
    x = x.replace(r"\t", r" ")
    x = x.replace(r"\b", r" ")
    x = x.replace(r"\r", r" ")
    x = re.sub(r"\s+", r" ", x)
    return x

def replace_n_punc(x):
  return re.sub(r'[\?\.\!\,\=]+(?=[\?\.\!\,\=])', '', x)
    
  
def _get_mispell(mispell_dict):
    mispell_re = re.compile('(%s)' % '|'.join(mispell_dict.keys()))
    return mispell_dict, mispell_re

mispellings, mispellings_re = _get_mispell(mispell_dict)
def replace_typical_misspell(text):
    def replace(match):
        return mispellings[match.group(0)]
    return mispellings_re.sub(replace, text)

In [0]:
df = pd.read_csv('/content/drive/My Drive/CCP-ICDsearch/desc-icd9-modeldata/model_data.csv')

train_X = train_df
test_X = test_df
# lower
train_X["comment_text"] = train_df["comment_text"].apply(lambda x: x.lower())
test_X["comment_text"] = test_df["comment_text"].apply(lambda x: x.lower())

# Clean speelings
train_X["comment_text"] = train_X["comment_text"].progress_apply(lambda x: replace_typical_misspell(x))
test_X["comment_text"] = test_X["comment_text"].apply(lambda x: replace_typical_misspell(x))
   
# Replcae multiple punct by single
train_X["comment_text"] = train_X["comment_text"].progress_apply(lambda x: replace_n_punc(x))
test_X["comment_text"] = test_X["comment_text"].apply(lambda x: replace_n_punc(x))    

# Clean numbers
train_X["comment_text"] = train_X["comment_text"].progress_apply(lambda x: clean_numbers(x))
test_X["comment_text"] = test_X["comment_text"].apply(lambda x: clean_numbers(x))

# Clean the text
train_X["comment_text"] = train_X["comment_text"].progress_apply(lambda x: clean_text(x))
test_X["comment_text"] = test_X["comment_text"].apply(lambda x: clean_text(x))

## fill up the missing values
train_X["comment_text"].fillna("_##_", inplace=True)
test_X["comment_text"].fillna("_##_", inplace=True)

In [0]:
#Tokenization
#train/test split

In [0]:
#Embedding matrix